In [1]:
import pandas as pd
import requests
import json
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import time
time.sleep(0.3)

In [33]:
df = pd.read_csv('data/2024(연간)/2024_사업보고서_03_포괄손익계산서_20251126.txt', sep='\t', encoding ='cp949')
df

,재무제표종류,종목코드,회사명,시장구분,업종,업종명,결산월,결산기준일,보고서종류,통화,항목코드,항목명,Unnamed: 12,당기,Unnamed: 14,Unnamed: 15,전기,전전기,Unnamed: 18
0,"단일 포괄손익계산서, 기능별 분류, 세후 - 별도",[060310],3S,코스닥시장상장법인,262,전자부품 제조업,3,2024-03-31,사업보고서,KRW,ifrs-full_OtherComprehensiveIncome,기타포괄손익,NaN,"-74,240,766",NaN,NaN,"-342,395,323",NaN,NaN
1,"단일 포괄손익계산서, 기능별 분류, 세후 - 별도",[060310],3S,코스닥시장상장법인,262,전자부품 제조업,3,2024-03-31,사업보고서,KRW,ifrs-full_OtherComprehensiveIncomeNetOfTaxGain...,확정급여제도의 재측정손익(세후기타포괄손익),NaN,"-74,240,766",NaN,NaN,"-342,395,323",NaN,NaN
2,"단일 포괄손익계산서, 기능별 분류, 세후 - 별도",[060310],3S,코스닥시장상장법인,262,전자부품 제조업,3,2024-03-31,사업보고서,KRW,dart_OperatingIncomeLoss,영업이익,NaN,"1,119,176,086",NaN,NaN,"2,105,054,604","1,153,521,366",NaN
3,"단일 포괄손익계산서, 기능별 분류, 세후 - 별도",[060310],3S,코스닥시장상장법인,262,전자부품 제조업,3,2024-03-31,사업보고서,KRW,dart_OtherGains,기타영업외이익,NaN,"3,030,723,370",NaN,NaN,"815,762,757","887,669,659",NaN
4,"단일 포괄손익계산서, 기능별 분류, 세후 - 별도",[060310],3S,코스닥시장상장법인,262,전자부품 제조업,3,2024-03-31,사업보고서,KRW,dart_OtherLosses,기타영업외비용,NaN,"670,744,208",NaN,NaN,"653,999,812","1,281,866,124",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50800,"단일 포괄손익계산서, 성격별 분류, 세전 - 별도",[238490],힘스,코스닥시장상장법인,292,특수 목적용 기계 제조업,12,2024-12-31,사업보고서,KRW,ifrs-full_GrossProfit,매출총이익,NaN,"14,539,110,262",NaN,NaN,"20,382,112,080","4,154,029,954",NaN
50801,"단일 포괄손익계산서, 성격별 분류, 세전 - 별도",[238490],힘스,코스닥시장상장법인,292,특수 목적용 기계 제조업,12,2024-12-31,사업보고서,KRW,ifrs-full_IncomeTaxExpenseContinuingOperations,법인세비용(수익),NaN,"-852,169,644",NaN,NaN,"454,516,466","-2,665,745,213",NaN
50802,"단일 포괄손익계산서, 성격별 분류, 세전 - 별도",[238490],힘스,코스닥시장상장법인,292,특수 목적용 기계 제조업,12,2024-12-31,사업보고서,KRW,ifrs-full_OtherComprehensiveIncome,기타포괄손익,NaN,"148,606",NaN,NaN,"2,617,929","-8,647,516",NaN
50803,"단일 포괄손익계산서, 성격별 분류, 세전 - 별도",[238490],힘스,코스닥시장상장법인,292,특수 목적용 기계 제조업,12,2024-12-31,사업보고서,KRW,ifrs-full_ProfitLoss,당기순이익(손실),NaN,"2,710,625,998",NaN,NaN,"5,599,495,130","-8,645,207,016",NaN


In [35]:
df = df[df['항목명'].str.contains('판관')]
df = df[['회사명', '항목명']]
df

,회사명,항목명
126,AP위성,"경상개발비, 판관비"
4837,TPC,"경상개발비, 판관비"
5469,경인전자,"경상개발비, 판관비"
8754,대림통상,"경상개발비, 판관비"
13568,리독스바이오,"경상개발비, 판관비"
18957,삼진,"경상개발비, 판관비"
34116,이글벳,"경상개발비, 판관비"
34903,이엘씨,"경상개발비, 판관비"
39838,케이아이엔엑스,"경상개발비, 판관비"
40035,케이엘넷,"경상개발비, 판관비"


In [29]:
df['항목명'].unique()

array(['         경상개발비, 판관비', '      경상개발비', '         연구개발비',
       '      연구개발비', '      경상연구개발비', '         연구개발비용', '      기술개발비',
       '      연구와 개발 비용', '         경상개발비', '         기술개발비',
       '            경상연구개발비', '         연구와 개발 비용', '         경상연구개발비',
       '         4.경상연구개발비'], dtype=object)

In [30]:
df.to_csv('개발회사.csv')